In [1]:
from gurobipy import *
import pandas as pd

In [2]:
df1_1 = pd.read_excel("OR110-1_case01.xlsx", "Instance 1")
# df1_2 = pd.read_excel("OR110-1_case01.xlsx", "Instance 2")
# df1_3 = pd.read_excel("OR110-1_case01.xlsx", "Instance 3")

df1_1 = df1_1.rename(columns={'Process Type': 'type_P1', 'Unnamed: 2': 'type_P2', 'Processing Time': 'time_P1', 'Unnamed: 5': 'time_P2'})
df1_1 = df1_1.drop(0)

df1_1['time_P2'] = df1_1['time_P2'].fillna(0)

display(df1_1)
#display(df1_2)
#display(df1_3)

,Job ID,type_P1,type_P2,Splitting Timing,time_P1,time_P2,Due Time
1,1.0,Boiling,Baking,1.0,2.7,1.3,12:30:00
2,2.0,Baking,Boiling,1.0,1.6,1.4,12:30:00
3,3.0,Boiling,Baking,1.0,0.7,1.9,12:30:00
4,4.0,Smoking,Boiling,1.0,0.5,0.7,12:30:00
5,5.0,Boiling,Baking,1.0,0.8,1.0,12:30:00
6,6.0,Boiling,NaN,0.0,2.5,0.0,12:30:00
7,7.0,Baking,Smoking,1.0,1.4,2.0,12:30:00
8,8.0,Baking,Smoking,1.0,1.1,1.1,17:30:00
9,9.0,Boiling,Baking,1.0,0.8,0.7,17:30:00
10,10.0,Baking,Boiling,1.0,1,0.5,17:30:00


In [3]:
n = len(df1_1["Job ID"])
J = range(n)
I = range(4)
# K = [(j, k) for j in J for k in J if j < k]
H = [(j, k) for j in J for k in J if j != k]

P = []
for i in J:
    P.append(df1_1["time_P1"][i+1] + df1_1["time_P2"][i+1])
    
display(P)

M1 = 0
for i in P:
    M1 += i

[4.0, 3.0, 2.5999999999999996, 1.2, 1.8, 2.5, 3.4, 2.2, 1.5, 1.5, 4.4, 2.0]

In [4]:
import datetime

start = datetime.time(7, 30, 0)
start = datetime.datetime.combine(datetime.date.today(), start)

D = []

for i in df1_1["Due Time"]:
    a = datetime.datetime.combine(datetime.date.today(), i) - start
    D.append(int(a.seconds / 3600))
    #due.append(datetime.combine(date.today(), i) - datetime.combine(date.today(), start))

print(D)

[5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10]


In [5]:
eg2 = Model("problem1")    # build a new model

x = eg2.addVars(I, J, lb = 0, vtype = GRB.BINARY, name = "x")
z = eg2.addVars(H, lb = 0, vtype = GRB.BINARY, name = "z")
t = eg2.addVars(J, lb = 0, vtype = GRB.BINARY, name = "t")
f = eg2.addVars(J, lb = 0, vtype = GRB.CONTINUOUS, name = "f")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-28


In [6]:
# setting the objective function 
eg2.setObjective(
    quicksum(t[j] for j in J)
    , GRB.MINIMIZE) 


# add constraints and name them

for j in J:
    eg2.addConstr((f[j] - D[j] <= M1 * t[j]), name = f"define t_{j+1}")
    
for j in J:
    eg2.addConstr(quicksum(x[i, j] for i in I) == 1, name = f"job {j+1} in one machine")
    
# for j, k in H:
#     eg2.addConstr(z[j, k] + z[k, j] <= 1, name = "z")

for j, k in H:
    for i in I:
        eg2.addConstr(z[j, k] + z[k, j] + 1 >= x[i, j] + x[i, k], name = f"define z_{j+1}_{k+1}")
#         eg2.addConstr(x[i, j] + x[i, k] <= 2, name = f"define x")
    
for j, k in H:
    eg2.addConstr(f[j] + P[k] - f[k] <= M1 * (1 - z[j, k]), name = f"define f_{k+1}")
#     eg2.addConstr(f[k] + P[j] - f[j] <= M1 * (1 - z[k, j]), name = f"define f_{j+1}")

for j in J:
    eg2.addConstr(f[j] >= P[j], name = f"f_{j+1} > P_{j+1}")
    
eg2.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 696 rows, 204 columns and 2592 nonzeros
Model fingerprint: 0x6567b863
Variable types: 12 continuous, 192 integer (192 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 6.0000000
Presolve removed 276 rows and 0 columns
Presolve time: 0.00s
Presolved: 420 rows, 204 columns, 1524 nonzeros
Variable types: 12 continuous, 192 integer (192 binary)

Root relaxation: objective 0.000000e+00, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    4    6.00000    0.00000   100%     -    0s
H    0     0                     

In [7]:
print("Result:")

# objecitve value
print("z* =", eg2.objVal)
print("")

# t
for j in J:
    print(f[j].varName, '=', round(f[j].x, 2), ', Due time =', D[j], ',', t[j].varName, '=', t[j].x)
print("")
    
# x
print("x:")
print("j \ i", end="")
print("\tMachine2 Machine3 Machine4 Machine5")
for j in J:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for i in I:
        # print values of each kind of product
        if len(str(x[i, j].x)) < 7:
            print(x[i, j].x, "\t", end="")
        else:
            print(y[i, j].x, "", end="")
    print("")    # use for change line
print("")

# z
print("z:")
print("j \ k", end="")
for j in J:
    print("\tJob" + str(j+1), end="")
print("")    # use for change line
for j in J:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for k in J:
        # print values of each kind of product
        if j != k:
            if len(str(z[j, k].x)) < 7:
                print(z[j, k].x, "\t", end="")
            else:
                print(z[j, k].x, "", end="")
        else:
            print("\t", end="")
    print("")    # use for change line
print("")

Result:
z* = 1.0

f[0] = 11.1 , Due time = 5 , t[0] = 1.0
f[1] = 4.5 , Due time = 5 , t[1] = -0.0
f[2] = 2.6 , Due time = 5 , t[2] = -0.0
f[3] = 1.2 , Due time = 5 , t[3] = -0.0
f[4] = 4.4 , Due time = 5 , t[4] = -0.0
f[5] = 4.9 , Due time = 5 , t[5] = -0.0
f[6] = 4.9 , Due time = 5 , t[6] = -0.0
f[7] = 7.1 , Due time = 10 , t[7] = -0.0
f[8] = 1.5 , Due time = 10 , t[8] = -0.0
f[9] = 1.5 , Due time = 10 , t[9] = -0.0
f[10] = 10.0 , Due time = 10 , t[10] = -0.0
f[11] = 8.3 , Due time = 10 , t[11] = -0.0

x:
j \ i	Machine2 Machine3 Machine4 Machine5
Job1 	-0.0 	1.0 	-0.0 	-0.0 	
Job2 	-0.0 	-0.0 	-0.0 	1.0 	
Job3 	-0.0 	-0.0 	1.0 	-0.0 	
Job4 	1.0 	-0.0 	-0.0 	-0.0 	
Job5 	-0.0 	-0.0 	1.0 	-0.0 	
Job6 	-0.0 	1.0 	-0.0 	-0.0 	
Job7 	1.0 	-0.0 	-0.0 	-0.0 	
Job8 	-0.0 	1.0 	-0.0 	-0.0 	
Job9 	-0.0 	-0.0 	-0.0 	1.0 	
Job10 	-0.0 	1.0 	-0.0 	-0.0 	
Job11 	-0.0 	-0.0 	1.0 	-0.0 	
Job12 	1.0 	-0.0 	-0.0 	-0.0 	

z:
j \ k	Job1	Job2	Job3	Job4	Job5	Job6	Job7	Job8	Job9	Job10	Job11	Job12
Job1 		-0.

In [8]:
eg3 = Model("problem1-2")    # build a new model

x = eg3.addVars(I, J, lb = 0, vtype = GRB.BINARY, name = "x")
z = eg3.addVars(J, J, lb = 0, vtype = GRB.BINARY, name = "z")
t = eg3.addVars(J, lb = 0, vtype = GRB.BINARY, name = "t")
f = eg3.addVars(J, lb = 0, vtype = GRB.CONTINUOUS, name = "f")

w = eg3.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

In [9]:
# setting the objective function 
eg3.setObjective(
    w
    , GRB.MINIMIZE) 


# add constraints and name them

for j in J:
    eg3.addConstr(w >= f[j], name="w")

eg3.addConstr(quicksum(t[j] for j in J) <= eg2.objVal, name = "limit tardy jobs")

for j in J:
    eg3.addConstr((f[j] - D[j] <= M1 * t[j]), name = f"define t_{j+1}")
    
for j in J:
    eg3.addConstr(quicksum(x[i, j] for i in I) == 1, name = f"job {j+1} in one machine")
    
# for j, k in H:
#     eg3.addConstr(z[j, k] + z[k, j] <= 1, name = "z")

for j, k in H:
    for i in I:
        eg3.addConstr(z[j, k] + z[k, j] + 1 >= x[i, j] + x[i, k], name = f"define z_{j+1}_{k+1}")
#         eg3.addConstr(x[i, j] + x[i, k] <= 2, name = f"define x")
    
for j, k in H:
    eg3.addConstr(f[j] + P[k] - f[k] <= M1 * (1 - z[j, k]), name = f"define f_{k+1}")
#     eg3.addConstr(f[k] + P[j] - f[j] <= M1 * (1 - z[k, j]), name = f"define f_{j+1}")

for j in J:
    eg3.addConstr(f[j] >= P[j], name = f"f_{j+1} > P_{j+1}")
    
eg3.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1369 rows, 217 columns and 3948 nonzeros
Model fingerprint: 0x11d24b81
Variable types: 13 continuous, 204 integer (204 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 870 rows and 12 columns
Presolve time: 0.01s
Presolved: 499 rows, 205 columns, 1692 nonzeros
Variable types: 13 continuous, 192 integer (192 binary)

Root relaxation: objective 4.400000e+00, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.40000    0   12          -    4.40000      -     -    0s
H    0     0                      10.0000000    4.40000  56.0%     -    0s
H 

In [10]:
print("Result:")

# objecitve value
print("z* =", round(eg3.objVal, 2))
print("")

# t
for j in J:
    print(f[j].varName, '=', round(f[j].x, 2), ', Due time =', D[j], ',', t[j].varName, '=', t[j].x)
print("")
    
# x
print("x:")
print("j \ i", end="")
print("\tMachine2 Machine3 Machine4 Machine5")
for j in J:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for i in I:
        # print values of each kind of product
        if len(str(x[i, j].x)) < 7:
            print(x[i, j].x, "\t", end="")
        else:
            print(y[i, j].x, "", end="")
    print("")    # use for change line
print("")

# z
print("z:")
print("j \ k", end="")
for j in J:
    print("\tJob" + str(j+1), end="")
print("")    # use for change line
for j in J:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for k in J:
        # print values of each kind of product
        if j != k:
            if len(str(z[j, k].x)) < 7:
                print(z[j, k].x, "\t", end="")
            else:
                print(z[j, k].x, "", end="")
        else:
            print("\t", end="")
    print("")    # use for change line
print("")

Result:
z* = 7.7

f[0] = 4.0 , Due time = 5 , t[0] = -0.0
f[1] = 3.0 , Due time = 5 , t[1] = 0.0
f[2] = 2.6 , Due time = 5 , t[2] = 0.0
f[3] = 3.8 , Due time = 5 , t[3] = -0.0
f[4] = 4.3 , Due time = 5 , t[4] = 0.0
f[5] = 2.5 , Due time = 5 , t[5] = -0.0
f[6] = 7.7 , Due time = 5 , t[6] = 1.0
f[7] = 7.7 , Due time = 10 , t[7] = 0.0
f[8] = 5.5 , Due time = 10 , t[8] = 0.0
f[9] = 5.5 , Due time = 10 , t[9] = 0.0
f[10] = 7.4 , Due time = 10 , t[10] = 0.0
f[11] = 7.5 , Due time = 10 , t[11] = 0.0

x:
j \ i	Machine2 Machine3 Machine4 Machine5
Job1 	1.0 	0.0 	0.0 	0.0 	
Job2 	-0.0 	1.0 	-0.0 	-0.0 	
Job3 	-0.0 	-0.0 	1.0 	-0.0 	
Job4 	-0.0 	-0.0 	1.0 	0.0 	
Job5 	-0.0 	-0.0 	-0.0 	1.0 	
Job6 	-0.0 	-0.0 	-0.0 	1.0 	
Job7 	0.0 	-0.0 	-0.0 	1.0 	
Job8 	0.0 	-0.0 	1.0 	0.0 	
Job9 	-0.0 	-0.0 	1.0 	0.0 	
Job10 	1.0 	-0.0 	0.0 	-0.0 	
Job11 	0.0 	1.0 	-0.0 	-0.0 	
Job12 	1.0 	-0.0 	-0.0 	0.0 	

z:
j \ k	Job1	Job2	Job3	Job4	Job5	Job6	Job7	Job8	Job9	Job10	Job11	Job12
Job1 		-0.0 	-0.0 	-0.0 	0.0 	-